기본 feature / pay_amt / ip_cnt_wrt_seq / seq_cnt_wrt_ip / phone_cnt_wrt_seq / avg_phone_cnt

2:1 undersampling, 5 bagging, validation 수정 (10월은 validation set)

categorical feature 일부 추가

In [1]:
import pandas as pd
from lightgbm import Booster
import lightgbm as lgb
import copy
import time
import json

In [2]:
train_path = '/home/workspace/user-workspace/slim_train.parquet'
test_path ='/home/workspace/user-workspace/slim_test.parquet'
encoder = '/home/workspace/user-workspace/cat_encoder.json'
decoder = '/home/workspace/user-workspace/inverse_cat_encoder.json'
data_dir = '/home/workspace/user-workspace/junheon/data/task150/'
model_dir = '/home/workspace/user-workspace/junheon/model/task150/'
result_dir = '/home/workspace/user-workspace/junheon/result/task150/'
submission_dir = '/home/workspace/user-workspace/prediction/'

In [3]:
bagging_size = 5
prefix = "ex6"

In [4]:
features = [
    "basic_feature",
    "month_day",
    "pay_amt",
    "ip_cnt_wrt_seq",
    "seq_cnt_wrt_ip",
    "phone_cnt_wrt_seq",
    "avg_phone_cnt",
    "first_days"
]

In [5]:
categorical_features = [
    "AGE", "COMMC_CLF", "NPAY_YN", "PAY_MTHD_CD", "ARS_AUTHTI_YN", "GNDR", "FOREI_YN",  "AUTHTI_CLF_FLG", 
    "SVC_CLF_NM", "CP_M_CLF_NM", "CP_S_CLF_NM", "month", "day"
]

# merge data

### train

In [6]:
for seed in range(bagging_size):
    df_list = []
    for name in features:
        feature_df = pd.read_parquet(f"{data_dir}{name}_{seed}.parquet").set_index("id")
        df_list.append(feature_df)
        pd.concat(df_list, axis=1).to_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")

### test

In [7]:
df_list = []
for name in features:
    feature_df = pd.read_parquet(f"{data_dir}{name}_test.parquet").set_index("id")
    df_list.append(feature_df)
    pd.concat(df_list, axis=1).to_parquet(f"{data_dir}{prefix}_test.parquet")

# Modeling

In [8]:
model_params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "num_leaves": 255,
    "max_depth": 8,
    "min_child_samples": 200,
    "subsample": 0.9,
    "subsample_freq": 1,
    "colsample_bytree": 0.5,
    "min_child_weight": 0,
    "subsample_for_bin": 1000000,
    "min_split_gain": 0,
    "reg_lambda": 0,
    "verbose": 0,
    "seed": 77
}

train_params = {
    "num_boost_round": 5000,
    "early_stopping_rounds": 30
}

### train and valid

In [9]:
boosters = []
train_results = []

In [10]:
for seed in range(bagging_size):
    start_time = time.time()
    
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")
    valid_df = train_df[train_df['month']==10]
    train_df = train_df[train_df['month']!=10]
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    valid_dataset = lgb.Dataset(valid_df.loc[:, valid_df.columns != "target"], label=valid_df["target"].values)
    eval_results = {}
    
    model: Booster = lgb.train(model_params,
                              train_dataset,
                              categorical_feature=categorical_features,
                              valid_sets=[train_dataset, valid_dataset],
                              valid_names=['train', 'valid'],
                              evals_result=eval_results,
                              **train_params)
        
    best_iteration = model.best_iteration
    
    params = copy.deepcopy(train_params)
    params["num_boost_round"] = best_iteration
    if "early_stopping_rounds" in params:
        del params["early_stopping_rounds"]
        
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    model = lgb.train(model_params,
                     train_dataset,
                     categorical_feature=categorical_features,
                     **params)
    
    model.save_model(f"{model_dir}{prefix}_model_{seed}.txt")
    boosters.append(model)
    
    result = dict()
    result['train_auc'] = eval_results['train']['auc'][best_iteration-1]
    result['valid_auc'] = eval_results['valid']['auc'][best_iteration-1]
    result['best_iteration'] = best_iteration
    result['train_time'] = time.time() - start_time
    result['feature_importance'] = {name: int(score) for name, score in zip(model.feature_name(), model.feature_importance())}
    result['feature_importance'] = {key: value for key, value in sorted(result['feature_importance'].items(), key=lambda item: item[1], reverse=True)}
    train_results.append(result)

/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['AGE', 'ARS_AUTHTI_YN', 'AUTHTI_CLF_FLG', 'COMMC_CLF', 'CP_M_CLF_NM', 'CP_S_CLF_NM', 'FOREI_YN', 'GNDR', 'NPAY_YN', 'PAY_MTHD_CD', 'SVC_CLF_NM', 'day', 'month']
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042939 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's auc: 0.961377	valid's auc: 0.960251
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's auc: 0.965119	valid's auc: 0.964206
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's auc: 0.968403	valid's auc: 0.967145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's auc: 0.96928	valid's auc: 0.969508
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's auc: 0.970379	valid's auc: 0.970574
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	train's auc: 0.970318	valid's auc: 0.970795
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	train's auc: 0.970763	valid's auc: 0.970935
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	

[68]	train's auc: 0.973535	valid's auc: 0.972744
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	train's auc: 0.97361	valid's auc: 0.97277
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	train's auc: 0.973685	valid's auc: 0.972819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	train's auc: 0.973656	valid's auc: 0.97276
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	train's auc: 0.973691	valid's auc: 0.972766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	train's auc: 0.973736	valid's auc: 0.972814
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	train's auc: 0.973708	valid's auc: 0.97275
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	train's auc: 0.973743	valid's auc: 0.972786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	train's

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [11]:
json.dump(train_results, open(f"{result_dir}{prefix}.result.json", 'w'), indent=2)

# Predict & Submission

In [12]:
test_df = pd.read_parquet(f"{data_dir}{prefix}_test.parquet")

In [13]:
submission = pd.read_parquet(test_path)[['TRD_NO']]

In [14]:
predictions = []

In [15]:
predictions = []
for i, booster in enumerate(boosters):
    print(f"prediction {i}")
    predictions.append(booster.predict(test_df).tolist())

In [16]:
for i, predict in enumerate(predictions):
    submission[f"target_{i}"] = predict

In [17]:
columns = [f"target_{i}" for i in range(len(predictions))]

In [18]:
columns

['target_0', 'target_1', 'target_2', 'target_3', 'target_4']

In [19]:
submission['target'] = submission[columns].mean(1)
submission = submission.drop(columns=columns)

In [20]:
submission.to_feather(f"{submission_dir}prediction.feather")

In [21]:
submission

,TRD_NO,target
0,nv97RPvpsILRkO0BLxDZgs8BHzilzEv8TzSWunbrJsA=,0.004645
1,81Qha5XTB9l/meAOl1KBPIqYcroUxJ1L4dJM7JpKZSA=,0.030763
2,Ie/OKGoMzkdBrgpcJUQ2nO4q9Z1ZzzdfPjOaq1yl+m8=,0.001399
3,ni1cHt1R9rxWJxcQO0g87zdrOod7EoJOWz78UES5jhY=,0.387166
4,foBvHXQ3rm8GZMveeXtzf/Yk5Wm9ar0Q73L/zM0Knw4=,0.059927
...,...,...
3910924,i9J3PUzIyFK/+2IX30dCv7fVW/xi+Viu2shFKhXr+dQ=,0.008281
3910925,t3jsnUnUZtJmqDNqbxyegFPYuxPWKDjf5i/7z2+dyto=,0.130045
3910926,eYUitVOB2+KIFr1HfBDRbxW+LS8GByrUtcKQnN0R5JU=,0.016145
3910927,PI27ia0T3JKe0oCQlTfkFUScoMSvmhr5c7SKrCFHTjU=,0.005422


In [22]:
from nipa.taskSubmit import nipa_submit
import os

team_id = "1390"
task_no= "150"
prediction_path = f"{submission_dir}prediction.feather"
# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
            task_no=task_no,
            result=prediction_path
           )


is file:  True
20201106204852409792_L7qj.feather: 200
